# Widgets
- Są to obiekty w Pythonie reagujące na zdarzenia i umożliwiające obsługę różnych popularnych kontrolek w przeglądarce. Wykorzystane zostaną do utworzenia prostego GUI do obsługi robota.

In [6]:
import ipywidgets as widgets

In [7]:
import rospy
from geometry_msgs.msg import Twist

In [8]:
rospy.init_node("widgets_controller")

- Na zajęciach w których omawiany był Publisher wysyłanie prędkości sterujących odbywało się w następuący sposób:

In [9]:
twist_publisher= rospy.Publisher("/turtle1/cmd_vel",Twist,queue_size=10)

some_message=Twist()
some_message.angular.z=1
some_message.linear.x=10

twist_publisher.publish(some_message)

In [10]:
def move_robot(forward_vel=5,rotation_vel=5):
    '''A function to move turtle from turtlesim simulation
    
    Args:
        forward_vel (float): Linear velocity
        rotation_vel (float): Angular velocity'''
    message=Twist()
    message.angular.z=rotation_vel
    message.linear.x=forward_vel
    
    
    twist_publisher.publish(message)

In [11]:
move_robot(1,-1)

# 1 ) Slider

- Jak widać zmiana prędkości odbywa się poprzez ręczne wprowadzenie wartośi i wywołanie funkcji. Z punktu widzenia użytkownika oczekiwałby on, aby mógł sterować robotem przy pomocy kontrolek. Wykorzystamy funkcję **move_robot** do utworzenia domyślnego widgetu sterującego prędkościami robota. Każda zmiana wartości kontrolki powoduje wywołanie funkcji **move_robot** i wysłanie odczytanych prędkości z kontrolek.

In [51]:
widgets.interact(move_robot)

interactive(children=(IntSlider(value=5, description='forward_vel', max=15, min=-5), IntSlider(value=5, descri…

<function __main__.move_robot(forward_vel=5, rotation_vel=5)>

- Można również zdefiniować własne kontrolki. Poniżej przykład z użyciem **FloatSlider** dla, którego ustawiane są wartości minimalne, maksymalne, krok zmiany oraz wartość początkowa. Jako pierwszy argument podawana jest nazwa funkcji, która ma zostać wywołana. Dla argumentów jakie przyjmuje funkcja *move_turtle* utworzone zostały odpowiednie slidery.

In [52]:
widgets.interact(move_robot,
                 forward_vel=widgets.FloatSlider(min=-10,max=10,step=2,value=0),
                 rotation_vel=widgets.FloatSlider(min=-3,max=3,value=0))

interactive(children=(FloatSlider(value=0.0, description='forward_vel', max=10.0, min=-10.0, step=2.0), FloatS…

<function __main__.move_robot(forward_vel=5, rotation_vel=5)>

#### Zadanie 1
Utworzyć slidery do zmiany koloru tła
1. Zaimportować odpowiednie biblioteki dla sersicu /clear
2. Utworzyć funkcję set_color przyjmującą 3 argumeny koloru, która ustawia odpowiednie wartości parametrów koloru tła. Było na zajęciach z ROS Parameter Server.
3. Wyczyścić serwisem */clear* tło mapy, aby możliwa była aktualizacja koloru.
4. Utworzyć widget z 3 sliderami ustawiającymi kolor.


In [18]:
from std_srvs.srv import Empty

In [19]:
def set_color(r=0,g=0,b=255):
    rospy.set_param("turtlesim/background_r", r)
    rospy.set_param("turtlesim/background_g", g)
    rospy.set_param("turtlesim/background_b", b)
    !rosservice call /clear

In [21]:
widgets.interact(set_color,
                 r=widgets.FloatSlider(min=0,max=255,step=2,value=0),
                 g=widgets.FloatSlider(min=0,max=255,step=2,value=0),
                 b=widgets.FloatSlider(min=0,max=255,step=2,value=0))

interactive(children=(FloatSlider(value=0.0, description='r', max=255.0, step=2.0), FloatSlider(value=0.0, des…

<function __main__.set_color(r=0, g=0, b=255)>

In [23]:
import matplotlib

@widgets.interact(rgb = widgets.ColorPicker(concise=True))
def kolor_tla(rgb):
    if rgb[0] != "#": #name of color
        rgb = matplotlib.colors.cnames[rgb]
    rgb = rgb.lstrip('#')
    r,g,b = tuple(int(rgb[i:i+2],16) for i in (0,2,4))
    print(r,g,b)

interactive(children=(ColorPicker(value='black', concise=True, description='rgb'), Output()), _dom_classes=('w…

# 2) Textbox
- Teraz czas na wysłanie dowolnej informacji na topicu **/informacja**. Skorzystanie z kontrolki do wprowadzania tekstu spowoduje, że każde wprowadzenie nowego znaku wyśle wiadomość. W terminalu można podejrzeć informację na tym topicu.

*rostopic echo /informacja*

Oto przykład:

In [12]:
from std_msgs.msg import String
def send_msg(wiadomosc):
    pub_info = pub_speed=rospy.Publisher("/informacja",String,queue_size=10)
    msg_info = String()
    msg_info.data = wiadomosc
    pub_info.publish(msg_info)

In [13]:
widgets.interact(send_msg,
                    wiadomosc=widgets.Text(value='Hello World!'))

interactive(children=(Text(value='Hello World!', description='wiadomosc'), Output()), _dom_classes=('widget-in…

<function __main__.send_msg(wiadomosc)>

# 3) Button

# 3.1)
- Jeśli chcielibyśmy wysłać dowolną informację po zakończeniu jej wprowadzania przydadlby się przycisk *wyślij* odpowiedzialny za przekazanie informacji.


In [59]:
widgets.ToggleButton(
    value=False,
    description='Click me',
    disabled=False,
    button_style='warning', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)

ToggleButton(value=False, button_style='warning', description='Click me', icon='check', tooltip='Description')

# 3.2)

In [31]:
input_text = widgets.Text(value='Hello World!', disabled=False, description="informacja:") #kontrolka
display(input_text)

Text(value='Hello World!', description='informacja:')

In [32]:
from std_msgs.msg import String
def send_msg(button_data):
    pub_info = rospy.Publisher("/informacja",String,queue_size=10)
    msg_info = String()
    msg_info.data = str(input_text.value)
    pub_info.publish(msg_info)

In [33]:
przycisk_wyslij = widgets.Button(
    value=False,
    description='wyslij',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)
przycisk_wyslij.on_click(send_msg)
display(przycisk_wyslij)

Button(description='wyslij', icon='check', style=ButtonStyle(), tooltip='Description')

[ERROR] [1619455243.735771]: bad callback: <function zmien_output at 0x7f2361c73160>
Traceback (most recent call last):
  File "/opt/ros/noetic/lib/python3/dist-packages/rospy/topics.py", line 750, in _invoke_callback
    cb(msg)
  File "<ipython-input-21-36c6af7ba19d>", line 2, in zmien_output
    output_tekst.value=wiadomosc.data
NameError: name 'output_tekst' is not defined

[ERROR] [1619455243.749388]: bad callback: <function zmien_output at 0x7f2348e89b80>
Traceback (most recent call last):
  File "/opt/ros/noetic/lib/python3/dist-packages/rospy/topics.py", line 750, in _invoke_callback
    cb(msg)
  File "<ipython-input-24-36c6af7ba19d>", line 2, in zmien_output
    output_tekst.value=wiadomosc.data
NameError: name 'output_tekst' is not defined

[ERROR] [1619455395.414996]: bad callback: <function zmien_output at 0x7f2361c73160>
Traceback (most recent call last):
  File "/opt/ros/noetic/lib/python3/dist-packages/rospy/topics.py", line 750, in _invoke_callback
    cb(msg)
  File "

- Odbior:

In [38]:
output_text = widgets.Text(value='Hello World!', disabled=False, description="informacja:") #kontrolka

display(output_text)

Text(value='Hello World!', description='informacja:')

In [39]:
def zmien_output(wiadomosc):
    output_text.value=wiadomosc.data
    
nasz_subscriber=rospy.Subscriber("/informacja",String,zmien_output)

In [37]:
nasz_subscriber.unregister()

- Wypisz pozycje:

In [42]:
def wypisz_pozycje_widget(pozycja):
    widget_x.value = str(pozycja.x)
    widget_y.value = str(pozycja.y)
    
widget_x=widgets.Text(value='no info', disabled=False, description="poz_x:")
widget_y=widgets.Text(value='no info', disabled=False, description="poz_y:")


In [43]:
from turtlesim.msg import Pose
pos_subscriber = rospy.Subscriber("turtle1/pose",Pose,wypisz_pozycje_widget)


In [44]:
display(widget_x)
display(widget_y)

Text(value='11.088889122009277', description='poz_x:')

Text(value='10.00434684753418', description='poz_y:')

# 3.3)
#### Zadanie 2
Utworzyć publishera i slidery do wysyłania prędkości postępowej i obrotowej do robota. Można skorzystać z dostępnych sliderów na początku tego skryptu.
Wiadomość o prędkościach powinna pojawiać się na topicu **/nazwa_robota/cmd_vel/slider**.

In [32]:
twist_publisher= rospy.Publisher("/turtle1/cmd_vel/slider",Twist,queue_size=10)

In [33]:
def move_robot(forward_vel=5,rotation_vel=5):

    message=Twist()
    message.angular.z=rotation_vel
    message.linear.x=forward_vel
    
    
    twist_publisher.publish(message)

In [34]:
widgets.interact(move_robot,
                 forward_vel=widgets.FloatSlider(min=-10,max=10,step=2,value=0),
                 rotation_vel=widgets.FloatSlider(min=-3,max=3,value=0))

interactive(children=(FloatSlider(value=0.0, description='forward_vel', max=10.0, min=-10.0, step=2.0), FloatS…

<function __main__.move_robot(forward_vel=5, rotation_vel=5)>

#### Zadanie 3
- Utworzyć dwie kontrolki do wprowadzania prędkości. Wysłanie prędkości powinno zostać zatwierdzone przez wciśnięcie przycisku. Można skorzystać z kontrolki **FloatText** przedstawionej poniżej. Można zauważyć, że zwracana wartość przez kontrolkę jest typu float i nie trzeba dokonać konwersji danych.

In [41]:
liczba = widgets.FloatText(
    value=7.5,
    description='Any:',
    disabled=False
)
display(liczba)
odczytana_liczba = liczba.value
print(odczytana_liczba)
print("typ wartości: ", type(odczytana_liczba))

FloatText(value=7.5, description='Any:')

7.5
typ wartości:  <class 'float'>


In [10]:
forward_vel=widgets.FloatText(
    value=7.5,
    description='Forward_vel = ',
    disabled=False
)
display(forward_vel)
rotation_vel=widgets.FloatText(
    value=7.5,
    description='Rotation_vel = ',
    disabled=False
)
display(rotation_vel)

FloatText(value=7.5, description='Forward_vel = ')

FloatText(value=7.5, description='Rotation_vel = ')

In [11]:
from geometry_msgs.msg import Twist

pub_info = rospy.Publisher("/turtle1/cmd_vel/button",Twist,queue_size=10)

def send_msg(button_data):
    msg_info = Twist()
    msg_info.linear.x = forward_vel.value
    msg_info.angular.z = rotation_vel.value
    pub_info.publish(msg_info)

In [12]:
!rosmsg info Twist

[geometry_msgs/Twist]:
geometry_msgs/Vector3 linear
  float64 x
  float64 y
  float64 z
geometry_msgs/Vector3 angular
  float64 x
  float64 y
  float64 z



In [13]:
!rostopic list

/rosout
/rosout_agg
/turtle1/cmd_vel
/turtle1/cmd_vel/button
/turtle1/color_sensor
/turtle1/pose


In [16]:
!rostopic info /turtle1/cmd_vel/button

Type: geometry_msgs/Twist

Publishers: 
 * /widgets_controller (http://localhost:35905/)

Subscribers: 
 * /rostopic_1311_1619423639561 (http://localhost:37641/)




In [17]:
!rosnode list

/rosout
/rostopic_1311_1619423639561
/turtlesim
/widgets_controller


In [15]:
przycisk_wyslij = widgets.Button(
    value=False,
    description='wyslij',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)
przycisk_wyslij.on_click(send_msg)
display(przycisk_wyslij)

Button(button_style='success', description='wyslij', icon='check', style=ButtonStyle(), tooltip='Description')

#### Zadanie 4
Utworzyć przycisk do wysyłania prędkości wraz z funkcją, która będzie wysyłała wartość z przycisków na topic **/nazwa_robota/cmd_vel/float_text**.

In [ ]:
# UZUPEŁNIĆ


#### Zadanie 5
Utworzyć parametr *select_controler* oraz uzupełnić poniższą klasę.

In [ ]:
rospy.set_param(..., ...) # UZUPEŁNIĆ ustawić parametr velocity_source, 1 - źródło slidery, 2 - float_text
class VelocityControler:
    def __init__(self):
        slider_sub = ... # UZUPEŁNIĆ subscriberem od topicu ze sliderów,  do odczytu wykorzystać 
                         # metodę klasy callback_slider. Odwołanie się do metod klasy następuje poprzez self.nazwa_metody w klasie.
        float_text_sub = ... # UZUPEŁNIĆ subscriberem od wartości prędkości wprowadzanych z pól tekstowych callback_float_text
        self.vel_pub = ... # UZUPEŁNIĆ utworzyć publishera, który będzie wysyłał wiadomości w zależności od ustawionego
                      # parametru velocity_source do robota. Gdzie wartość parametru 1 oznacza, że źródłem jest
                      # slider, a wartość 2 - float_text. Dla pozostałych wartości parametru robot stoi w miejscu.
                      # Wysyłana jest prędkość 0 zatrzymująca robota. Wiadomość powinna zostać opublikowana na topic
                      # sterującym robotem /nazwa_robota/cmd_vel
        
    def callback_slider(self, msg_data):
        # UZUPEŁNIĆ zweryfikować stan parametru i jeśli tryb jest właściwy przekazywać prędkości sterujące
        ...
        
    def callback_float_text(self, msg_data):
        # UZUPEŁNIĆ zweryfikować stan parametru i jeśli tryb jest właściwy przekazywać prędkości sterujące
        ...

In [ ]:
VelocityControler()

# 4) Checkbox

In [45]:
random_checkbox=widgets.Checkbox(
    value=False,
    description='Check me',
    disabled=False,
    indent=False
)

In [46]:
random_checkbox

Checkbox(value=False, description='Check me', indent=False)

#### Zadanie 6
Odczytać wartość z przycisku i ustawić parametr **velocity_source**.

In [ ]:
# UZUPEŁNIĆ

def set_velocity_source(slider_source): 
    #UZUPEŁNIĆ
    ...
    
    
widgets.interact(set_velocity_source,
                slider_source = ... # UZUPEŁNIĆ
                )

# 5) Tabs

Do grupowania zakladek i wyświetlania wszystkich w jednej karcie służy **VBox** z omawianej biblioteki. Przyjmuje listę kontrolek do wyświetlenia.

In [48]:
tab_contents = ['P0', 'P1', 'P2']
children = [widgets.VBox([widgets.Text(description=name),
                          widgets.IntSlider(),
                          widgets.IntSlider(),
                          widgets.IntSlider()])
            for name in tab_contents]

tab = widgets.Tab(children = children)
for i in range(len(tab_contents)):
    tab.set_title(i, tab_contents[i])
tab

In [49]:
tab.children

(VBox(children=(Text(value='', description='P0'), IntSlider(value=0), IntSlider(value=0), IntSlider(value=0))),

In [54]:
kontrolka1 = widgets.IntSlider(description="speed")
kontrolka1_2 = widgets.Checkbox(description="slider_source")
kontrolka2 = widgets.IntSlider(description="cos tam")
kontrolka3 = widgets.Text(description="info")
kontrolka3_2 = widgets.IntSlider(description="forward")
kontrolka3_3 = widgets.IntSlider(description="rotational")

children = [
    widgets.VBox([kontrolka1, kontrolka1_2]),
    widgets.VBox([kontrolka2]),
    widgets.VBox([kontrolka3, kontrolka3_2, kontrolka3_3])
]

tab = widgets.Tab(children = children)
# ustawianie nazwy zakładek; Kolejne argumenty: id zakładki, nazwa zakładki
tab.set_title(0, "P1")
tab.set_title(1, "P2")
tab.set_title(2, "P3")
tab

In [52]:
?? kontrolka1.on_trait_change

In [ ]:
?? kontrolka1.observe

In [53]:
wyn=kontrolka1.observe(print)

In [55]:
def zaw_suw(slownik_wart):
    print(f"aktualna zawartosc suwaka to {slownik_wart['new']}")

<function print>

### Dodatkowe widgety
https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html
### Zmiana stylu i układu kontrolek
https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Styling.html